In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # on mac

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from utils.mag_scripts import * 
from utils.barplots import * 


import altair as alt


# supress warnings
import warnings
warnings.filterwarnings('ignore')

Metagenome-atlas output summary
===============================

Her is the code to create the output summary

In [ ]:
#go to output file 
%cd ../Example/

## Metagenome assembled genomes

### Taxonomy

In [ ]:
Tax= pd.read_table('Results/taxonomy.tsv',index_col=0)
Tax

In [ ]:
# create a short label for each species
Labels=Tax.ffill(axis=1).species.copy()
Labels.loc[Tax.species.isnull()]+= ' '+ Labels.index[Tax.species.isnull()]

### Genome quality 

In [ ]:
genome_quality= pd.read_table('Results/genome_completeness.tsv',index_col=0)

genome_quality['Quality_Score']= genome_quality.eval('Completeness -5*Contamination')
genome_quality['Lineage']= genome_quality['Marker lineage'].map(lambda s: s.split()[0])

genome_quality['Id']= genome_quality.index

genome_quality= genome_quality.join(Tax)
genome_quality['Name']= Labels

In [ ]:

xscale = alt.Scale(domain=(0, 15))
yscale = alt.Scale(domain=(50, 101))

alt.Chart(genome_quality).mark_circle(opacity= .6).encode(
    alt.X('Contamination', scale=xscale),
    alt.Y('Completeness', scale=yscale),
    color='phylum',
    tooltip=['Name', 'Id', 'Contamination','Completeness' ]
).interactive()

## Abundance

In [ ]:
Counts= pd.read_csv('Results/counts/raw_counts_genomes.tsv',index_col=0,sep='\t').T
Counts.head()

### Mapping rate

In [ ]:
mapping_rate = pd.read_table('Results/mapping_rate.tsv',index_col=0,squeeze=True)
f,ax= plt.subplots(figsize=(2,4))
ax.set_ylim([0,1])
sns.swarmplot(y= mapping_rate,ax=ax)

ax.set_title('Mapping rate')


### Stats based on raw Counts

There are good reasons to use rawcounts and use centric log ratios, see more in 

Gloor, Gregory B., Jean M. Macklaim, Vera Pawlowsky-Glahn, and Juan J. Egozcue. 2017. “Microbiome Datasets Are Compositional: And This Is Not Optional.” Frontiers in Microbiology 8 (November). Frontiers: 2224. doi:10.3389/fmicb.2017.02224.


For differencial abundance analysis see also the same paper.

In [ ]:
# transforme counts with centrig log ratio

data= clr(Counts)


In [ ]:
sns.clustermap(data.T,
            row_cluster=True,cmap='RdBu_r', center=0,
             yticklabels= Labels,
              )


### PCA (PCoA) of the robust aitchison distance

In [ ]:
from sklearn.decomposition import PCA

pca= PCA()
transformed_data= pca.fit_transform(data)


In [ ]:
pca_data= pd.DataFrame()
pca_data['PC 1']= transformed_data[:,0]
pca_data['PC 2']= transformed_data[:,1]
pca_data['Sample']= data.index



alt.Chart(pca_data).mark_circle(size=60).encode(
    x='PC 1',
    y='PC 2',
    tooltip=['Sample' ]
).interactive()


## Relative abundance


For the relative abundance we take the coverage over the genome not the raw counts. This inmplicit normalizes for genome size. The coverage is calculated as the median of the coverage values calculated in 1kb blocks.

In [ ]:
D = pd.read_table("Results/counts/median_coverage_genomes.tsv",index_col=0)
D.head()

In [ ]:
#calculate relative abundance

relab = (D.T/D.sum(1)).T

In [ ]:
# get most abundant genomes

counts_per_genome= relab.sum().sort_values()
ax= counts_per_genome[-10:].plot.bar(figsize=(10,5))

_= ax.set_xticklabels(Labels.loc[counts_per_genome.index[-10:]])
ax.set_title('Most abundant genomes')
ax.set_ylabel('Abundance [relab]')

### Typical bar chart

In [ ]:

level='family'

grouped_data =relab.groupby(Tax[level],axis=1).sum()

ax= BarPlot(grouped_data)

ax.legend_.set_title(level,{'weight':'bold'})


## Functional annotation


Relative abundance of functional annotations per sample

The abundance is calculated as the sum of the relative abundance of all bacteria containing a function.

### CAZy

In [ ]:
#CAZy
CAZy_annotations_genome= pd.read_table('Results/annotations/CAZy.tsv',index_col=0)
CAZy_presence= (CAZy_annotations_genome>0).astype(int)
CAZy_presence.head()


function_relab = relab @ CAZy_presence

sns.clustermap(function_relab)

function_relab.head()

### Kegg orthologs

In [ ]:
#Kegg orthologs

Kegg_annotations_genome= pd.read_table('Results/annotations/KO.tsv',index_col=0)
Kegg_presence= (Kegg_annotations_genome>0).astype(int)
Kegg_presence.head()


function_relab = relab @ Kegg_presence

sns.clustermap(function_relab)

function_relab.head()